In [43]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
    "administrations,moyens généraux",
    "comptabilite,finance",
   "metiers banque et assurances",
   "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
   "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
     "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "commercial,technico commercial,service client",
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(120)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(120)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

df_offers


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E25FD6A]
	(No symbol) [0x00007FF73E25FC20]
	(No symbol) [0x00007FF73E2799A1]
	(No symbol) [0x00007FF73E3021B7]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUserThreadStart [0x00007FF92370AA58+40]


In [ ]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers

In [32]:
df_offers['Provided Text'][0]

"Description de l’organisationmyAgro est une entreprise sociale évoluant dans le domaine agricole. Nous intervenons au Mali, au Sénégal et en Côte d’Ivoire. myAgro vise à améliorer les conditions de vie des petits producteurs en milieu rural à sortir de la pauvreté. myAgro permet aux petits agriculteurs de se financer en payant petit à petit en avance pour les intrants. Nous utilisons les téléphones portables – un producteur peut payer pour les intrants en utilisant le même système d’achat d’unités pour son portable. En plus, myAgro s’engage dans le domaine du développement communautaire en menant des actions de sensibilisation.Notre vision et objectif à travers notre Étoile polaire sont de travailler avec 1 millions de petits producteurs à l’horizon 2026. Pour réaliser cet objectif ambitieux, myAgro cherche à étendre ses activités à toute l'Afrique de l'Ouest.Titre du poste : Field Coordinator (FC)Contrat : CDDDisponibilité : immédiateLocation : le poste de Field Coordinator (FC) sera

# direction générale,direction d'unité

In [42]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Liste des liens pour chaque catégorie
categories = [
    "administrations,moyens généraux",
   "assistanat,secrétariat",
    "comptabilite,finance",
   "metiers banque et assurances",
   "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
   "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
     "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "vente,televente,assistanat",
    "Qualité,sécurité,Environnement"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list




df_offers


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E25FD6A]
	(No symbol) [0x00007FF73E25FC20]
	(No symbol) [0x00007FF73E2799A1]
	(No symbol) [0x00007FF73E3021B7]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUserThreadStart [0x00007FF92370AA58+40]


In [ ]:
# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]
df_offers

# Métiers de l'agriculture

In [38]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
   #"toutes les offres d'emploi",
   
    
    
    "journalisme,médias,traduction"
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)


# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Lieu de travail', "Date d'expiration", 'Niveau de poste', "Secteur d'activité",
    "Nombre de postes", "Provided Text"
]]

# Afficher le DataFrame
df_offers



,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Nombre de postes,Provided Text
0,BTL Specialist,Exceliam,"Abidjan, Côte d'ivoire",28 Décembre 2023,Confirmé / Expérimenté,1 à 2 ans,0,"Abidjan, Côte d'ivoire",27 Mars,Confirmé / Expérimenté| Jeune diplômé,Services,01,L’objectif principal de ce poste est de mettre...
1,Mobile Developer Android & iOS,HELP 24 COTE D'IVOIRE,Côte d'ivoire,16 Novembre 2023,Débutant / Junior,1 à 2 ans,0,Côte d'ivoire,14 Février,Débutant / Junior,"Informatique, Télécom, Internet",01,Nous recherchons un développeur mobile junior ...


# Hotellerie,Tourisme,Restauration, Loisirs

In [22]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie


categories = [
    #"toutes les offres d'emploi",
    "journalisme,médias,traduction",
   
    "vente,televente,assistanat",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
     "autre",
    "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)


# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Lieu de travail', "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text"
]]

# Afficher le DataFrame
df_offers


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text
0,Boy Cuisinier,Exceliam,"Abidjan, Côte d'ivoire",04 Décembre 2023,Confirmé / Expérimenté,6 à 10 ans,0,"Abidjan, Côte d'ivoire",03 Mars,Confirmé / Expérimenté,Services,TS Bac +2| Formation Professionnelle,01,CDI,Le métier de cuisinier consiste à confectionne...
1,Gestionnaire Food Court,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,Le FOOD COURT est un restaurant qui permettra ...
2,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...
3,Commercial Junior (hôtellerie) H/F,YESHI GROUP,Côte d'ivoire,08 Janvier,Débutant / Junior,3 à 5 ans,0,"Côte d'ivoire| Abidjan, Côte d'ivoire",07 Avril,Débutant / Junior,"BTP, Construction, Immobilier",NaN,01,CDI| CDD,Missions du posteDéfinir et identifier les cib...


In [18]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
    #"toutes les offres d'emploi",
    #"administrations,moyens généraux",
   #"assistanat,secrétariat",
    #"comptabilite,finance",
   #1"metiers banque et assurances",
   #1"juridique,fiscal,audit,conseil",
   #1 "RH,personnel,formation",
   #1 "education,enseignement",
   #"direction générale,direction d'unité",
   #1 "vente,televente,assistanat",
   # "commercial,technico commercial,service client",
    #1"responsable commercial,grands comptes",
    #1"créatio, design",
   #1 "marketing, communication",
    #1 "journalisme,médias,traduction",
    #1"informatique,systèmes d'information,internet",          
   #1 "télécommunication,réseaux",
   #1 "chantier,métiers BTP,architecture",
   #1 "ingénierie,etudes,projet,R&D",
    #1 "logistique,achat,stock,transport",
   #1 "production,méthode,industrie",
   # "maintenance,entretien",
    #"Qualité,sécurité,Environnement",
    ##"Santé,Médical,Pharmacie",
    #"Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
   # "autre",              
   ## "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers



KeyError: "None of [Index(['Intitulé du poste', 'Nom de l'entreprise', 'Lieu de travail',\n       'Date d'expiration', 'Niveau de poste', 'Secteur d'activité',\n       'Niveau d'étude (diplome)', 'Nombre de postes', 'Type de contrat',\n       'Provided Text'],\n      dtype='object')] are in the [columns]"

In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
    #"toutes les offres d'emploi",
    #"administrations,moyens généraux",
   #"assistanat,secrétariat",
    #"comptabilite,finance",
   #1"metiers banque et assurances",
   #1"juridique,fiscal,audit,conseil",
   #1 "RH,personnel,formation",
   #1 "education,enseignement",
   #"direction générale,direction d'unité",
   # "vente,televente,assistanat",
   # "commercial,technico commercial,service client",
    #"responsable commercial,grands comptes",
    #"créatio, design",
   # "marketing, communication",
    #"journalisme,médias,traduction",
    #1"informatique,systèmes d'information,internet",          
   # "télécommunication,réseaux",
   # "chantier,métiers BTP,architecture",
   # "ingénierie,etudes,projet,R&D",
    #"logistique,achat,stock,transport",
    #"production,méthode,industrie",
   # "maintenance,entretien",
    #"Qualité,sécurité,Environnement",
    #"Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    #"Ouvriers qualifiés, Chauffeur",
   # "autre",              
   # "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers



In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
    #"toutes les offres d'emploi",
    #"administrations,moyens généraux",
   #"assistanat,secrétariat",
    #"comptabilite,finance",
   #1"metiers banque et assurances",
   #1"juridique,fiscal,audit,conseil",
   #1 "RH,personnel,formation",
   #1 "education,enseignement",
   #"direction générale,direction d'unité",
   # "vente,televente,assistanat",
   # "commercial,technico commercial,service client",
    #"responsable commercial,grands comptes",
    #"créatio, design",
   # "marketing, communication",
    #"journalisme,médias,traduction",
    #1"informatique,systèmes d'information,internet",          
   # "télécommunication,réseaux",
   # "chantier,métiers BTP,architecture",
   # "ingénierie,etudes,projet,R&D",
    #"logistique,achat,stock,transport",
    #"production,méthode,industrie",
   # "maintenance,entretien",
    #"Qualité,sécurité,Environnement",
    #"Santé,Médical,Pharmacie",
    #"Hotellerie,Tourisme,Restauration, Loisirs",
    #"Ouvriers qualifiés, Chauffeur",
    "autre",              
   # "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers



In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie

categories = [
    #"toutes les offres d'emploi",
    #"administrations,moyens généraux",
   #"assistanat,secrétariat",
    #"comptabilite,finance",
   #1"metiers banque et assurances",
   #1"juridique,fiscal,audit,conseil",
   #1 "RH,personnel,formation",
   #1 "education,enseignement",
   #"direction générale,direction d'unité",
   # "vente,televente,assistanat",
   # "commercial,technico commercial,service client",
    #"responsable commercial,grands comptes",
    #"créatio, design",
   # "marketing, communication",
    #"journalisme,médias,traduction",
    #1"informatique,systèmes d'information,internet",          
   # "télécommunication,réseaux",
   # "chantier,métiers BTP,architecture",
   # "ingénierie,etudes,projet,R&D",
    #"logistique,achat,stock,transport",
    #"production,méthode,industrie",
   # "maintenance,entretien",
    #"Qualité,sécurité,Environnement",
    #"Santé,Médical,Pharmacie",
    #"Hotellerie,Tourisme,Restauration, Loisirs",
    #"Ouvriers qualifiés, Chauffeur",
    #"autre",              
    "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(2)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers



In [6]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Liste des liens pour chaque catégorie
categories = [
    #"toutes les offres d'emploi",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "comptabilite,finance",
    "metiers banque et assurances",
    "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    #"autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# Créer un DataFrame avec les informations de base
basic_info_df = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})

# Afficher le DataFrame
basic_info_df.head()


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0
1,Administrateur Systèmes et Réseaux,Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",03 Janvier,Confirmé / Expérimenté,3 à 5 ans,0
2,Acheteur - H/F,Société Générale Côte D'ivoire,Côte d'ivoire,18 Octobre 2023,Confirmé / Expérimenté,6 à 10 ans,0
3,HRBP Senior – H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,0
4,Directeur d'Agence (h/f),BICICI,"Abidjan, Côte d'ivoire",23 Novembre 2023,Confirmé / Expérimenté,3 à 5 ans,0


In [7]:
# Suite de la deuxième partie du script

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Construire une URL absolue
            absolute_url = urljoin(base_url, offre_link)

            # Accéder à la page de l'offre pour collecter plus d'informations
            try:
                driver.get(absolute_url)
                # Augmenter le délai d'attente pour laisser le temps à la page de se charger
                element_present = EC.presence_of_element_located((By.ID, 'some_element_id'))
                WebDriverWait(driver, 400).until(element_present)
                time.sleep(5)  # Attendre un peu plus après le chargement de la page
            except TimeoutException:
                print(f"Délai d'attente dépassé pour le chargement de la page ({absolute_url})")
                continue  # Passer à l'itération suivante en cas de délai d'attente dépassé

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                try:
                    for li in details_section.find_all('li', class_='row-fluid'):
                        key = li.find('span', class_='span4').text.strip()
                        value = li.find('span', class_='span8').text.strip()
                        job_details[key] = value
                except AttributeError as e:
                    print(f"Erreur lors de l'extraction des détails de l'offre ({absolute_url}): {e}")
                    continue  # Passer à l'itération suivante en cas d'erreur

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_offers = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplôme)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

# Afficher le DataFrame
df_offers


Délai d'attente dépassé pour le chargement de la page (https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135180-consultant-manager-general)


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E25FD6A]
	(No symbol) [0x00007FF73E25FC20]
	(No symbol) [0x00007FF73E2799A1]
	(No symbol) [0x00007FF73E3021B7]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUserThreadStart [0x00007FF92370AA58+40]


In [ ]:
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, ElementNotInteractableException

import dateparser
import requests
import os
#RECEIVERS = ['2250779535153','0711038318','22507095000','2250708008655','2250758895661','2250798382448','2250759573239','2250747208005','2250777654457']
RECEIVERS = ['2250779535153']
def send_sms(msg: str,title: str='QLICKVIEW', receivers: list=RECEIVERS):
    # msg += " Priere de mener des actions"
    msg = msg.replace(' ', '+')
    for receiver in receivers:
        os.system(f'''curl -X GET "http://10.1.60.190:9802/dispatcher/httpconnectserver/smsaffaires.jsp?UserName=application_smsaffaires_sva&Password=sva&SenderAppId=1&DA={receiver}&SOA={title}&Flags=264192&Content={msg}"''')



def check_server_availability(url, retry_interval=10, max_retries=3):
    """
    Vérifie la disponibilité d'un serveur en effectuant une requête GET vers l'URL donnée.

    Paramètres :
    url (str) : L'URL du serveur à vérifier.
    retry_interval (int) : Intervalle en secondes entre chaque tentative (par défaut : 10 secondes).
    max_retries (int) : Le nombre maximum de tentatives avant de considérer le serveur comme indisponible (par défaut : 3).

    Retourne :
    bool : True si le serveur est disponible, False sinon.
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                print(f"Le serveur à l'URL {url} est disponible.")
                #send_sms(f"Le serveur à l'URL {url} est disponible.")
                return True
            else:
                print(f"Le serveur à l'URL {url} a renvoyé un code de réponse : {response.status_code}")
                send_sms(f"Le serveur à l'URL {url} a renvoyé un code de réponse : {response.status_code}")
        except requests.ConnectionError:
            print(f"Impossible de se connecter au serveur à l'URL {url}. Tentative {attempt + 1} sur {max_retries}...")
            send_sms(f"Impossible de se connecter au serveur à l'URL {url}. Tentative {attempt + 1} sur {max_retries}...")
        time.sleep(retry_interval)
    
    print(f"Le serveur à l'URL {url} est indisponible après {max_retries} tentatives.")
    send_sms(f"Le serveur à l'URL {url} est indisponible après {max_retries} tentatives.")
    return False

def connect_to_url(driver_path, url):
    try:
        # Configuration du service du navigateur Firefox pour être rapide et robuste
        #service = Service(driver_path)
        #service = driver_path

        # Configuration des options pour le navigateur Firefox
        options = webdriver.FirefoxOptions()
        
        options.binary_location = r'C:\\Program Files\\Mozilla Firefox\\firefox.exe'

        options.headless = True  # Exécution sans affichage de la fenêtre du navigateur
        options.page_load_strategy = 'normal'  # Attendre que la page soit complètement chargée

        # Création du pilote du navigateur Firefox
        driver = webdriver.Firefox(executable_path=driver_path, options=options)

        # Accès à l'URL fournie
        driver.get(url)

        # Retourne le pilote du navigateur
        return driver
    except Exception as e:
        print("Une erreur s'est produite lors de la connexion au navigateur ou de l'accès à l'URL.")
        send_sms("Une erreur s'est produite lors de la connexion au navigateur ou de l'accès à l'URL.")
        print(e)
        return None
'''def connect_to_url(driver_path, url, options):
    
    # Création d'une instance driver
    driver = webdriver.Firefox(executable_path=driver_path, options = options)
    #connexion à l'url
    driver.get(url)
    
    return driver'''
def authentication(driver, login="OCITNETAD\\admqv", password="@Ql1kV13w@"):
    try:
        # Recherche des éléments de saisie de l'identifiant et du mot de passe
        enter_login = driver.find_element(By.ID, "username")
        enter_password = driver.find_element(By.ID, "password")
        
        # Saisie de l'identifiant et du mot de passe
        enter_login.clear()
        enter_login.send_keys(login)
        time.sleep(5)

        enter_password.clear()
        enter_password.send_keys(password)
        time.sleep(5)

        # Soumission du formulaire d'authentification
        driver.find_element(By.CLASS_NAME, "loginSubmitButton").submit()
        time.sleep(5)

        return driver
    except Exception as e:
        print("Une erreur s'est produite lors de l'authentification.")
        send_sms("Une erreur s'est produite lors de l'authentification.")
        print(e)
        return None
def get_service_app(driver, service_name, id_):
    try:
        waiting_time = 10

        # Rafraîchissement de la page pour s'assurer que nous sommes sur la page principale
        driver.refresh()
        time.sleep(5)
        
        # Recherche de l'élément de saisie de recherche et saisie du nom du service à rechercher
        input_search = driver.find_element(By.CLASS_NAME, "input_search")
        input_search.clear()
        input_search.send_keys(service_name + Keys.RETURN)
        time.sleep(5)

        # Clic sur le premier résultat (nom du service) pour accéder à la page détaillée du dashboard
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", driver.find_element(By.CLASS_NAME, "name"))
            driver.find_element(By.CLASS_NAME, "name").click()
        except ElementClickInterceptedException:
            # Si l'élément intercepte le clic, essayons de cliquer sur l'icône de flèche pour réduire le panneau
            driver.find_element(By.CLASS_NAME, "icon-chevron-down").click()
            time.sleep(2)
            driver.execute_script("arguments[0].scrollIntoView(true);", driver.find_element(By.CLASS_NAME, "name"))
            driver.find_element(By.CLASS_NAME, "name").click()

        time.sleep(waiting_time)

        try:
            # Attente de l'élément spécifié (date) sur la page détaillée du dashboard
            WebDriverWait(driver, waiting_time).until(
                EC.presence_of_element_located((By.ID, id_))
            )
            # Récupération du texte (date) de l'élément spécifié
            text = driver.find_element(By.ID, id_).find_element(By.CLASS_NAME, "TextObject").text
            
            # Retour à la page principale après la récupération de la date
            driver.back()
        except:
            # Si l'élément n'est pas trouvé, on rafraîchit la page et réessaie
            driver.refresh()
            WebDriverWait(driver, waiting_time).until(
                EC.presence_of_element_located((By.ID, id_))
            )
            text = driver.find_element(By.ID, id_).find_element(By.CLASS_NAME, "TextObject").text
            
            # Retour à la page principale après la récupération de la date
            driver.back()

        return text
    except TimeoutException:
        print(f"Timeout : Élément avec l'ID {id_} de {service_name} non trouvé. Veuillez vérifier l'ID.")
        send_sms(f"Timeout : Élément avec l'ID {id_} de {service_name} non trouvé. Veuillez vérifier l'ID.")
        return None
    except Exception as e:
        print("Une erreur s'est produite lors de la recherche des dashboards et de la récupération des dates.")
        #send_sms("Une erreur s'est produite lors de la recherche des dashboards et de la récupération des dates.")
        print(e)
        return None
def get_update_dates(driver, dashboards_info):
    try:
        # Dictionnaire pour stocker les dates de mise à jour des dashboards
        update_dates = dict()
        
        # Parcours des noms de services et de leurs IDs correspondants dans le dictionnaire dashboards_info
        for device, id_ in dashboards_info.items():
            # Appel de la fonction pour récupérer la date du dashboard
            text = get_service_app(driver, service_name=device, id_=id_)
            if text is not None:
                # Stockage de la date de mise à jour dans le dictionnaire
                update_dates[device.replace(".qvw", "")] = text
        
        # Fermeture du navigateur une fois que toutes les dates ont été récupérées
        driver.quit()
        return update_dates
    except Exception as e:
        print("Une erreur s'est produite lors du traitement des informations collectées.")
        send_sms("Une erreur s'est produite lors du traitement des informations collectées.")
        print(e)
        return None

def extract_date_from_text(text):
    try:
        thedate = text.split(':')
        thedate = thedate[-1]
        return thedate.strip()
    except Exception as e:
        print("Une erreur s'est produite lors de l'extraction de la date du texte.")
        send_sms("Une erreur s'est produite lors de l'extraction de la date du texte.")
        print(e)
        return None

def transform_string_date(str_val):
    try:
        date_ = dateparser.parse(str_val, languages=["fr"])
        if date_:
            return date_.strftime("%Y-%m-%d")
        else:
            print(f"La date {str_val} n'a pas pu être transformée.")
            send_sms(f"La date {str_val} n'a pas pu être transformée.")
            return None
    except Exception as e:
        print("Une erreur s'est produite lors de la transformation de la date.")
        #send_sms("Une erreur s'est produite lors de la transformation de la date.")
        print(e)
        return None

# ... (le reste du script reste inchangé) ...
    
def save_data(devices_infos, dirPath='/data2'):
    try:
        # Création d'un DataFrame pandas avec les noms des applications et les dates de mise à jour
        df_dates = pd.DataFrame(
            {"applications": devices_infos.keys(), "dates_MAJ": devices_infos.values()},
            columns=["applications", "dates_MAJ"]
        )

        # Extraction et transformation des dates de mise à jour
        df_dates.dates_MAJ = df_dates.dates_MAJ.apply(lambda x: extract_date_from_text(x))
        df_dates.dates_MAJ = df_dates.dates_MAJ.apply(lambda x: transform_string_date(x))

        # Filtrer les lignes contenant des valeurs manquantes (NaN) dans les dates
        df_missing_dates = df_dates[df_dates['dates_MAJ'].isna()]
        if not df_missing_dates.empty:
            print("Lignes contenant des valeurs manquantes, veillez alerter pour le traitement:")
            send_sms("Lignes contenant des valeurs manquantes, veillez alerter pour le traitement:")
            print(df_missing_dates)
        # Supprimer les lignes contenant des valeurs manquantes (NaN)
        df_dates.dropna(subset=['dates_MAJ'], inplace=True)  
        # Obtention de la date actuelle pour le nom du fichier CSV
        today = datetime.now().strftime("%Y%m%d_%H")
        
        # Enregistrement des données dans un fichier CSV
        df_dates.to_csv(dirPath+'/niveau_maj.csv', index=False, header=None, sep=',')
        df_dates.to_csv(dirPath+'/checking_dashboards_'+today+'.csv', index=False,header=None, sep=';')

        return ''
    except Exception as e:
        print("Une erreur s'est produite lors de la sauvegarde des informations sur les mises à jour.")
        send_sms("Une erreur s'est produite lors de la sauvegarde des informations sur les mises à jour.")
        print(e)
        return None

if _name_ == "_main_":
    start_time = time.time()
    # Vérification de la disponibilité du serveur
    url_du_serveur = "http://10.242.69.36/qlikview/FormLogin.htm"  # Remplacez cette URL par l'URL de votre serveur
    serveur_disponible = check_server_availability(url_du_serveur)

    if serveur_disponible:
        # Affectations de valeur aux variables
        dash_path = '../configs/liste_dashboards.json'
        dirPath = ".\\datasources"
        driver_path =  "C:\\WORKING_SPACES\\WORKSPACE_BRICE\\TDB_Qlick_View_BO\\.wdm\\drivers\\geckodriver\\win64\\0.32\\geckodriver.exe"

        url = "http://10.242.69.36/qlikview/FormLogin.htm"

        dashboards_info = {
            "Orange-TV.qvw": "24",
            "Reporting DRDI.qvw": "16",
            "Reporting DEVICES.qvw": "29",
            "Reporting SVA.qvw": "59",
            "Money Report.qvw": "38",
            "Dashboard Recouvrement_V3.qvw": "69",
            "Acte de Gestion": "42",
            "REPORTING_CG_FRA.qvw": "11",
            "REPORTING_CG_FRA_VANTAGE.qvw": "33" #16
        }

        driver = connect_to_url(driver_path=driver_path, url=url)
        if driver is not None:
            driver = authentication(driver=driver, login="OCITNETAD\\admqv", password="FN7B%qy6@Kz2w9*")
            if driver is not None:
                update_dates = get_update_dates(driver=driver, dashboards_info=dashboards_info)
                if update_dates is not None:
                    save_data(devices_infos=update_dates, dirPath=dirPath)

        # Enregistrez l'heure actuelle après l'exécution du script
        end_time = time.time()
        # Calculer le temps pris par le script
        execution_time = end_time - start_time
        print(f"Le script s'est exécuté en {execution_time:.2f} secondes.")
        #send_sms(f"Le script s'est exécuté en {execution_time:.2f} secondes.")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=comptabilit%C3%A9',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=finance',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=statistique',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=gestion-de-projets'
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []  
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête
    
    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# Convertir les offres d'emploi en DataFrame
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})

# Afficher le DataFrame
# Mapping dictionary for experience classes

experience_mapping = {
    'Moins d’un an': 1,
    'Sans expérience' : 1,
    '1 à 2 ans': 2,
    '3 à 5 ans': 4,
    '6 à 10 ans': 8,
    'Plus de 10 ans': 15  # Assuming "Plus de 10 ans" corresponds to more than 10 years
}
# Apply the mapping to the 'Experience' column
df_offers['Experience'] = df_offers['Experience_lettre'].map(experience_mapping)

# Display the updated DataFrame
# Apply the mapping to the 'Experience' column
df_offers['Experience'] = df_offers['Experience_lettre'].map(experience_mapping)
#Ajout de la colonne ID
df_offers['ID'] = range(1, len(df_offers) + 1)
df_offers.insert(0, 'ID', df_offers.pop('ID'))

# Create quartile bins
# Créer les intervalles de quartile
quartile_bins = pd.qcut(df_offers['Experience'], q=4, labels=False, duplicates='drop')

# Créer la colonne 'Classe'
# Créer la colonne 'Classe' avec seulement trois labels
df_offers['Classe'] = pd.qcut(df_offers['Experience'], q=4, labels=['Q1', 'Q2', 'Q3'], duplicates='drop')



# Display the updated DataFrame
df_offers



In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def extract_job_details(job_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(job_url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    return job_details

# Example usage:
job_url = 'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire'
details = extract_job_details(job_url)
df = pd.DataFrame([details])
df

,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat
0,Côte d'ivoire,04 Avril,Responsable d'équipe| Confirmé / Expérimenté,"Informatique, Télécom, Internet","Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDD


In [14]:
import time
import re

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=comptabilit%C3%A9',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=finance',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=statistique',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=gestion-de-projets'
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []  
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête
    
    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# Convertir les offres d'emploi en DataFrame
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Technicien de Traitement des Eaux (H/F),SOLIBRA (Société de Limonaderies et de Brasser...,"Abidjan, Côte d'ivoire",Hier,Confirmé / Expérimenté,1 à 2 ans,0
1,Comptable Paie (H/F),SOLIBRA (Société de Limonaderies et de Brasser...,"Abidjan, Côte d'ivoire",Hier,Confirmé / Expérimenté,1 à 2 ans,0
2,Stagiaire Technicien de laboratoire (BOUAFLE),SOLIBRA (Société de Limonaderies et de Brasser...,"Bouafle, Côte d'ivoire",Avant hier,Débutant / Junior,Moins d’un an,0
3,Commercial Junior (hôtellerie) H/F,YESHI GROUP,Côte d'ivoire,08 Janvier,Débutant / Junior,3 à 5 ans,0
4,Superviseur Marketing Opérationnel,KOMIAN AI,Côte d'ivoire,05 Janvier,Responsable d'équipe,3 à 5 ans,0
5,Assistant Social Entreprise - H/F,Société Générale Côte D'ivoire,Côte d'ivoire,05 Janvier,Confirmé / Expérimenté,3 à 5 ans,0
6,Stagiaire Assistante de Direction,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",04 Janvier,Stagiaire / Etudiant,Moins d’un an,0
7,Télévendeur/Téléconseiller,OLSY GROUP,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0
8,Stagiaire Professionnel Assistant (e) Marketing,Entreprise anonyme,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0
9,Développeur Informatique,AKWABA TECHNOLOGIES SOLUTIONS,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,1 à 2 ans,0


In [17]:
import time
import re

# Liste des liens pour chaque catégorie
category_links = [

    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []  
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête
    
    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# Convertir les offres d'emploi en DataFrame
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers





,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Superviseur Marketing Opérationnel,KOMIAN AI,Côte d'ivoire,05 Janvier,Responsable d'équipe,3 à 5 ans,0
1,Stagiaire Professionnel Assistant (e) Marketing,Entreprise anonyme,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0
2,BTL Specialist,Exceliam,"Abidjan, Côte d'ivoire",28 Décembre 2023,Confirmé / Expérimenté,1 à 2 ans,0
3,Stagiaire Data Scientist,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",01 Décembre 2023,Jeune diplômé,Moins d’un an,0
4,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,0
5,Community Manger,KOMIAN AI,Côte d'ivoire,10 Décembre 2023,Confirmé / Expérimenté,3 à 5 ans,0
6,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0
7,Un(e) Commercial,SI’CLEAN PRESSING,Côte d'ivoire,09 Novembre 2023,Débutant / Junior,1 à 2 ans,0
8,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
9,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0


In [31]:
print(df_offers)


                                           Intitule  \
0                Superviseur Marketing Opérationnel   
1   Stagiaire Professionnel Assistant (e) Marketing   
2                                    BTL Specialist   
3                          Stagiaire Data Scientist   
4             Stagiaire Gestion de Projets Digitaux   
5                                  Community Manger   
6                        Consultant Manager Général   
7                                  Un(e) Commercial   
8                             Gestionnaire Tech Bar   
9                     Gestionnaire Centre Financier   
10                          Gestionnaire Food Court   
11               Chargé d'Affaires Entreprise Trade   
12               Commercial Junior (hôtellerie) H/F   
13                     Chargé de missions RSE (H/F)   
14            Chargé d'Affaires Trade Finance - H/F   

                                           Entreprise                    Pays  \
0                                     

In [24]:
print(df)
                          Lieu de travail Date d'expiration  \
0                           Côte d'ivoire          04 Avril   
1                  Abidjan, Côte d'ivoire        12 Janvier   
2                  Abidjan, Côte d'ivoire           27 Mars   
3                  Abidjan, Côte d'ivoire        29 Février   
4                  Abidjan, Côte d'ivoire        27 Février   
5                           Côte d'ivoire           09 Mars   
6                  Abidjan, Côte d'ivoire        04 Février   
7                           Côte d'ivoire        07 Février   
8                  Abidjan, Côte d'ivoire        04 Février   
9                  Abidjan, Côte d'ivoire        04 Février   
10                 Abidjan, Côte d'ivoire        04 Février   
11                 Abidjan, Côte d'ivoire        08 Février   
12  Côte d'ivoire| Abidjan, Côte d'ivoire          07 Avril   
13                 Abidjan, Côte d'ivoire           03 Mars   
14                          Côte d'ivoire           03 Mars   

                                 Niveau de poste  \
0   Responsable d'équipe| Confirmé / Expérimenté   
1                              Débutant / Junior   
2          Confirmé / Expérimenté| Jeune diplômé   
3                                  Jeune diplômé   
4                              Débutant / Junior   
5      Débutant / Junior| Confirmé / Expérimenté   
6              Manager / Responsable département   
7                              Débutant / Junior   
8   Responsable d'équipe| Confirmé / Expérimenté   
9   Responsable d'équipe| Confirmé / Expérimenté   
10  Responsable d'équipe| Confirmé / Expérimenté   
11                        Confirmé / Expérimenté   
12                             Débutant / Junior   
13                             Débutant / Junior   
14                        Confirmé / Expérimenté   

                 Secteur d'activité  \
0   Informatique, Télécom, Internet   
1     BTP, Construction, Immobilier   
2                          Services   
3        Banque, Assurance, Finance   
4        Banque, Assurance, Finance   
5   Informatique, Télécom, Internet   
6                          Services   
7                          Services   
8                          Services   
9                          Services   
10                         Services   
11       Banque, Assurance, Finance   
12    BTP, Construction, Immobilier   
13       Banque, Assurance, Finance   
14                       Industries   

                             Niveau d'étude (diplome)  \
0   Licence (LMD), Bac + 3| Master 1, Licence  Bac...   
1                              Licence (LMD), Bac + 3   
2   Licence (LMD), Bac + 3| Master 1, Licence  Bac...   
3                       Master 2, Ingéniorat, Bac + 5   
4                       Master 2, Ingéniorat, Bac + 5   
5                                                 NaN   
6   Master 1, Licence  Bac + 4| Master 2, Ingénior...   
7                                                 NaN   
8                          Master 1, Licence  Bac + 4   
9                          Master 1, Licence  Bac + 4   
10                         Master 1, Licence  Bac + 4   
11  Master 1, Licence  Bac + 4| Master 2, Ingénior...   
12                                                NaN   
13                                                NaN   
14  Master 1, Licence  Bac + 4| Master 2, Ingénior...   

                          Nombre de postes                    Type de contrat  \
0                                       01                                CDD   
1   01                        poste ouvert  Stage – Temps partiel| Pré-emploi   
2                                       01             Indépendant/Saisonnier   
3                                       01              Stage – Temps partiel   
4                                       01              Stage – Temps partiel   
5                                       01                                NaN   
6                                       01                         Consultant   
7                                       01                                NaN   
8                                       01                           CDI| CDD   
9                                       01                           CDI| CDD   
10                                      01                           CDI| CDD   
11                                      01                           CDI| CDD   
12                                      01                           CDI| CDD   
13                                      01                           CDI| CDD   
14  01                        poste ouvert                           CDI| CDD   

                                        Provided Text  \
0   KOMIAN IA est une entreprise innovante dans le...   
1   Intervenant dans le secteur de l’industrie chi...   
2   L’objectif principal de ce poste est de mettre...   
3   Nous recrutons pour le Hub Marketing AFS de So...   
4   La Société Générale Afrique de l'Ouest recherc...   
5   Si vous êtes un Community Manager inspiré, cré...   
6   Notre client une société qui vient de se const...   
7   Missions  Prospecter et démarcher les potentie...   
8   Le TECH BAR est un coworking qui permettra aux...   
9   Le TECH BAR est un coworking qui permettra aux...   
10  Le CENTRE FINANCIER est une unité au sein de l...   
11  La banque en toute confiance.  Acteur majeur d...   
12  Missions du posteDéfinir et identifier les cib...   
13  La banque en toute confiance.  Acteur majeur d...   
14  ENTITE :                                      ...   

               Nom de l'entreprise  
0                              NaN  
1               Entreprise anonyme  
2                              NaN  
3                              NaN  
4                              NaN  
5                              NaN  
6                              NaN  
7                              NaN  
8                              NaN  
9                              NaN  
10                             NaN  
11                             NaN  
12                             NaN  
13                             NaN  
14  Société Générale Côte D'ivoire  

                          Lieu de travail Date d'expiration  \
0                           Côte d'ivoire          04 Avril   
1                  Abidjan, Côte d'ivoire        12 Janvier   
2                  Abidjan, Côte d'ivoire           27 Mars   
3                  Abidjan, Côte d'ivoire        29 Février   
4                  Abidjan, Côte d'ivoire        27 Février   
5                           Côte d'ivoire           09 Mars   
6                  Abidjan, Côte d'ivoire        04 Février   
7                           Côte d'ivoire        07 Février   
8                  Abidjan, Côte d'ivoire        04 Février   
9                  Abidjan, Côte d'ivoire        04 Février   
10                 Abidjan, Côte d'ivoire        04 Février   
11                 Abidjan, Côte d'ivoire        08 Février   
12  Côte d'ivoire| Abidjan, Côte d'ivoire          07 Avril   
13                 Abidjan, Côte d'ivoire           03 Mars   
14                          Côte d'ivoire           03 

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def extract_job_details(job_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(job_url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    return job_details

# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
 "https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135362-stagiaire-professionnel-assistant-e-marketing",
  "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135344-btl-specialist-abidjan-cote-d-ivoire", 
  "https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135270-stagiaire-data-scientist-abidjan-cote-d-ivoire",
  "https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135260-stagiaire-gestion-de-projets-digitaux-abidjan-cote-d-ivoire",  # Add more job URLs here
  "https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135299-community-manger-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135180-consultant-manager-general-abidjan-cote-d-ivoire", 
   "https://www.novojob.com/cote-d-ivoire/entreprise/si-clean-pressing/offres-d-emploi/offre-d-emploi/135204-un-e-commercial-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire",
    "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135184-gestionnaire-centre-financier-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135208-charge-d-affaires-entreprise-trade-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/yeshi-group/offres-d-emploi/offre-d-emploi/135377-commercial-junior-hotellerie-h-f-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135274-charge-de-missions-rse-h-f-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f" 
]

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    details = extract_job_details(url)
    all_job_details.append(details)

# Create a DataFrame
df = pd.DataFrame(all_job_details)
df


,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,Côte d'ivoire,04 Avril,Responsable d'équipe| Confirmé / Expérimenté,"Informatique, Télécom, Internet","Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDD,KOMIAN IA est une entreprise innovante dans le...,NaN
1,"Abidjan, Côte d'ivoire",12 Janvier,Débutant / Junior,"BTP, Construction, Immobilier","Licence (LMD), Bac + 3",01 poste ouvert,Stage – Temps partiel| Pré-emploi,Intervenant dans le secteur de l’industrie chi...,Entreprise anonyme
2,"Abidjan, Côte d'ivoire",27 Mars,Confirmé / Expérimenté| Jeune diplômé,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,Indépendant/Saisonnier,L’objectif principal de ce poste est de mettre...,NaN
3,"Abidjan, Côte d'ivoire",29 Février,Jeune diplômé,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,Nous recrutons pour le Hub Marketing AFS de So...,NaN
4,"Abidjan, Côte d'ivoire",27 Février,Débutant / Junior,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,La Société Générale Afrique de l'Ouest recherc...,NaN
5,Côte d'ivoire,09 Mars,Débutant / Junior| Confirmé / Expérimenté,"Informatique, Télécom, Internet",NaN,01,NaN,"Si vous êtes un Community Manager inspiré, cré...",NaN
6,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
7,Côte d'ivoire,07 Février,Débutant / Junior,Services,NaN,01,NaN,Missions Prospecter et démarcher les potentie...,NaN
8,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
9,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN


In [ ]:
import time
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd

def extract_job_details(job_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(job_url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    return job_details

# Your existing code for scraping job listings
# ...

# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
    # Add more job URLs here
]

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    details = extract_job_details(url)
    all_job_details.append(details)

# Create a DataFrame for job listings
df_listings = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})

# Create a DataFrame for job details
df_details = pd.DataFrame(all_job_details)

# Merge the two DataFrames on the 'Lien' column
df_combined = pd.merge(df_listings, df_details, on='Lien')

# Display the combined DataFrame
print(df_combined)


In [69]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)
  
# Nouvelle liste pour stocker tous les détails des emplois
all_job_details = []


# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135362-stagiaire-professionnel-assistant-e-marketing',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135344-btl-specialist-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135270-stagiaire-data-scientist-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135260-stagiaire-gestion-de-projets-digitaux-abidjan-cote-d-ivoire',  # Add more job URLs here
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135299-community-manger-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135180-consultant-manager-general-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/si-clean-pressing/offres-d-emploi/offre-d-emploi/135204-un-e-commercial-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135184-gestionnaire-centre-financier-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135208-charge-d-affaires-entreprise-trade-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/yeshi-group/offres-d-emploi/offre-d-emploi/135377-commercial-junior-hotellerie-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135274-charge-de-missions-rse-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f' 
]

for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df = pd.DataFrame(all_job_details)



# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Intitule'] = intitules_list
df['Entreprise'] = entreprises_list
df['Pays'] = pays_list
df['Date'] = dates_list
df['Niveau'] = niveau_list
df['Experience_lettre'] = experience_list
df['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df = df[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien',"Lieu de travail","Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

print(df)


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,Superviseur Marketing Opérationnel,KOMIAN AI,Côte d'ivoire,05 Janvier,Responsable d'équipe,3 à 5 ans,0,Côte d'ivoire,04 Avril,Responsable d'équipe| Confirmé / Expérimenté,"Informatique, Télécom, Internet","Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDD,KOMIAN IA est une entreprise innovante dans le...,NaN
1,Stagiaire Professionnel Assistant (e) Marketing,Entreprise anonyme,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0,"Abidjan, Côte d'ivoire",Aujourd'hui,Débutant / Junior,"BTP, Construction, Immobilier","Licence (LMD), Bac + 3",01 poste ouvert,Stage – Temps partiel| Pré-emploi,Intervenant dans le secteur de l’industrie chi...,Entreprise anonyme
2,BTL Specialist,Exceliam,"Abidjan, Côte d'ivoire",28 Décembre 2023,Confirmé / Expérimenté,1 à 2 ans,0,"Abidjan, Côte d'ivoire",27 Mars,Confirmé / Expérimenté| Jeune diplômé,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,Indépendant/Saisonnier,L’objectif principal de ce poste est de mettre...,NaN
3,Stagiaire Data Scientist,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",01 Décembre 2023,Jeune diplômé,Moins d’un an,0,"Abidjan, Côte d'ivoire",29 Février,Jeune diplômé,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,Nous recrutons pour le Hub Marketing AFS de So...,NaN
4,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,0,"Abidjan, Côte d'ivoire",27 Février,Débutant / Junior,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,La Société Générale Afrique de l'Ouest recherc...,NaN
5,Community Manger,KOMIAN AI,Côte d'ivoire,10 Décembre 2023,Confirmé / Expérimenté,3 à 5 ans,0,Côte d'ivoire,09 Mars,Débutant / Junior| Confirmé / Expérimenté,"Informatique, Télécom, Internet",NaN,01,NaN,"Si vous êtes un Community Manager inspiré, cré...",NaN
6,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
7,Un(e) Commercial,SI’CLEAN PRESSING,Côte d'ivoire,09 Novembre 2023,Débutant / Junior,1 à 2 ans,0,Côte d'ivoire,07 Février,Débutant / Junior,Services,NaN,01,NaN,Missions Prospecter et démarcher les potentie...,NaN
8,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
9,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN


In [70]:
print(df)

                                           Intitule  \
0                Superviseur Marketing Opérationnel   
1   Stagiaire Professionnel Assistant (e) Marketing   
2                                    BTL Specialist   
3                          Stagiaire Data Scientist   
4             Stagiaire Gestion de Projets Digitaux   
5                                  Community Manger   
6                        Consultant Manager Général   
7                                  Un(e) Commercial   
8                             Gestionnaire Tech Bar   
9                     Gestionnaire Centre Financier   
10                          Gestionnaire Food Court   
11               Chargé d'Affaires Entreprise Trade   
12               Commercial Junior (hôtellerie) H/F   
13                     Chargé de missions RSE (H/F)   
14            Chargé d'Affaires Trade Finance - H/F   

                                           Entreprise                    Pays  \
0                                     

In [ ]:
li.separator-bot:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)
                    
                    
                <div class="left-bloc text-center" itemprop="logo" itemtype="http://schema.org/Organization">
                    <img src="https://www.novojob.com/cote-d-ivoire/attachments/company_logo/logo_2544359_large.jpg" alt="KOMIAN AI - Superviseur Marketing Opérationnel" width="80" height="50" class="has-radius">
                </div>
               <div class="row-fluid">
                   
                  <a title="Superviseur Marketing Opérationnel" href="https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/135370-superviseur-marketing-operationnel">
                    <h2 class="ellipsis row-fluid">
                        Superviseur Marketing Opérationnel                    </h2>
                  </a>  
                                        <div class="contact">
                     <h6 class="ellipsis"><span>
                         KOMIAN AI                        </span></h6>
                    </div>
                                    </div>
                <div onclick="document.location.href = 'https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/135370-superviseur-marketing-operationnel'; return false" class="row-fluid">
                    <div class="bloc-bottom phone-text-small">
                        <span class="spaced-right"><span><span><i class="fa fa-map-marker icon-left"></i>Côte d'ivoire</span></span></span>                                                <span class="spaced-right"><i class="fa fa-clock-o icon-left"></i> 05 Janvier</span>
                                                <span class="spaced-right phone-display-blok"><i class="fa fa-bookmark icon-left"></i> Responsable d'équipe (<span>3 à 5 ans</span>)</span>
                                            </div>
                </div>
            

In [48]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)


def extract_job_details(job_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(job_url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    return job_details

# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
 "https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135362-stagiaire-professionnel-assistant-e-marketing",
  "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135344-btl-specialist-abidjan-cote-d-ivoire", 
  "https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135270-stagiaire-data-scientist-abidjan-cote-d-ivoire",
  "https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135260-stagiaire-gestion-de-projets-digitaux-abidjan-cote-d-ivoire",  # Add more job URLs here
  "https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135299-community-manger-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135180-consultant-manager-general-abidjan-cote-d-ivoire", 
   "https://www.novojob.com/cote-d-ivoire/entreprise/si-clean-pressing/offres-d-emploi/offre-d-emploi/135204-un-e-commercial-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire",
    "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135184-gestionnaire-centre-financier-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135208-charge-d-affaires-entreprise-trade-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/yeshi-group/offres-d-emploi/offre-d-emploi/135377-commercial-junior-hotellerie-h-f-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135274-charge-de-missions-rse-h-f-abidjan-cote-d-ivoire",
   "https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f" 
]

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    details = extract_job_details(url)
    all_job_details.append(details)

# Create a DataFrame
df = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Intitule'] = intitules_list
df['Entreprise'] = entreprises_list
df['Pays'] = pays_list
df['Date'] = dates_list
df['Niveau'] = niveau_list
df['Experience_lettre'] = experience_list
df['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df = df[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien',"Lieu de travail","Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

df


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,Superviseur Marketing Opérationnel,KOMIAN AI,Côte d'ivoire,05 Janvier,Responsable d'équipe,3 à 5 ans,0,Côte d'ivoire,04 Avril,Responsable d'équipe| Confirmé / Expérimenté,"Informatique, Télécom, Internet","Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDD,KOMIAN IA est une entreprise innovante dans le...,NaN
1,Stagiaire Professionnel Assistant (e) Marketing,Entreprise anonyme,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0,"Abidjan, Côte d'ivoire",12 Janvier,Débutant / Junior,"BTP, Construction, Immobilier","Licence (LMD), Bac + 3",01 poste ouvert,Stage – Temps partiel| Pré-emploi,Intervenant dans le secteur de l’industrie chi...,Entreprise anonyme
2,BTL Specialist,Exceliam,"Abidjan, Côte d'ivoire",28 Décembre 2023,Confirmé / Expérimenté,1 à 2 ans,0,"Abidjan, Côte d'ivoire",27 Mars,Confirmé / Expérimenté| Jeune diplômé,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,Indépendant/Saisonnier,L’objectif principal de ce poste est de mettre...,NaN
3,Stagiaire Data Scientist,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",01 Décembre 2023,Jeune diplômé,Moins d’un an,0,"Abidjan, Côte d'ivoire",29 Février,Jeune diplômé,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,Nous recrutons pour le Hub Marketing AFS de So...,NaN
4,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,0,"Abidjan, Côte d'ivoire",27 Février,Débutant / Junior,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,Stage – Temps partiel,La Société Générale Afrique de l'Ouest recherc...,NaN
5,Community Manger,KOMIAN AI,Côte d'ivoire,10 Décembre 2023,Confirmé / Expérimenté,3 à 5 ans,0,Côte d'ivoire,09 Mars,Débutant / Junior| Confirmé / Expérimenté,"Informatique, Télécom, Internet",NaN,01,NaN,"Si vous êtes un Community Manager inspiré, cré...",NaN
6,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
7,Un(e) Commercial,SI’CLEAN PRESSING,Côte d'ivoire,09 Novembre 2023,Débutant / Junior,1 à 2 ans,0,Côte d'ivoire,07 Février,Débutant / Junior,Services,NaN,01,NaN,Missions Prospecter et démarcher les potentie...,NaN
8,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
9,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN


In [37]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135362-stagiaire-professionnel-assistant-e-marketing',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135344-btl-specialist-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135270-stagiaire-data-scientist-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135260-stagiaire-gestion-de-projets-digitaux-abidjan-cote-d-ivoire',  # Add more job URLs here
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135299-community-manger-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135180-consultant-manager-general-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/si-clean-pressing/offres-d-emploi/offre-d-emploi/135204-un-e-commercial-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135184-gestionnaire-centre-financier-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135208-charge-d-affaires-entreprise-trade-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/yeshi-group/offres-d-emploi/offre-d-emploi/135377-commercial-junior-hotellerie-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135274-charge-de-missions-rse-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f' 
]

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df = pd.DataFrame(all_job_details)
df


                          Lieu de travail Date d'expiration  \
0                           Côte d'ivoire          04 Avril   
1                  Abidjan, Côte d'ivoire        12 Janvier   
2                  Abidjan, Côte d'ivoire           27 Mars   
3                  Abidjan, Côte d'ivoire        29 Février   
4                  Abidjan, Côte d'ivoire        27 Février   
5                           Côte d'ivoire           09 Mars   
6                  Abidjan, Côte d'ivoire        04 Février   
7                           Côte d'ivoire        07 Février   
8                  Abidjan, Côte d'ivoire        04 Février   
9                  Abidjan, Côte d'ivoire        04 Février   
10                 Abidjan, Côte d'ivoire        04 Février   
11                 Abidjan, Côte d'ivoire        08 Février   
12  Côte d'ivoire| Abidjan, Côte d'ivoire          07 Avril   
13                 Abidjan, Côte d'ivoire           03 Mars   
14                          Côte d'ivoire           03 

In [68]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# URL of the job category page
category_link = 'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing'

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Visiter la page des offres d'emploi
driver.get(category_link)
time.sleep(5)  # Attendre 5 secondes avant de récupérer le code source

# Utiliser BeautifulSoup pour extraire les liens des offres d'emploi
soup = BeautifulSoup(driver.page_source, 'html.parser')
job_links = [a['href'] for a in soup.find_all('a', href=True, class_='ellipsis row-fluid')]

# Fermer le pilote Selenium à la fin
driver.quit()

# Afficher la liste des liens des offres d'emploi
job_links


[]

In [ ]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

# Nouvelle liste pour stocker tous les détails des emplois
all_job_details = []


# Example usage for multiple job URLs
job_urls = [
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135370-superviseur-marketing-operationnel-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi/offre-d-emploi/cote-d-ivoire/abidjan/135362-stagiaire-professionnel-assistant-e-marketing',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135344-btl-specialist-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135270-stagiaire-data-scientist-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/societe-generale-afrique-de-l-ouest-centre-d-expertise-afs/offres-d-emploi/offre-d-emploi/135260-stagiaire-gestion-de-projets-digitaux-abidjan-cote-d-ivoire',  # Add more job URLs here
    'https://www.novojob.com/cote-d-ivoire/entreprise/komian-ai/offres-d-emploi/offre-d-emploi/135299-community-manger-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135180-consultant-manager-general-abidjan-cote-d-ivoire', 
    'https://www.novojob.com/cote-d-ivoire/entreprise/si-clean-pressing/offres-d-emploi/offre-d-emploi/135204-un-e-commercial-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135181-gestionnaire-tech-bar-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/excelliam/offres-d-emploi/offre-d-emploi/135184-gestionnaire-centre-financier-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135208-charge-d-affaires-entreprise-trade-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/yeshi-group/offres-d-emploi/offre-d-emploi/135377-commercial-junior-hotellerie-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/entreprise/bicici/offres-d-emploi/offre-d-emploi/135274-charge-de-missions-rse-h-f-abidjan-cote-d-ivoire',
    'https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f' 
]

for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df = pd.DataFrame(all_job_details)



# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Intitule'] = intitules_list
df['Entreprise'] = entreprises_list
df['Pays'] = pays_list
df['Date'] = dates_list
df['Niveau'] = niveau_list
df['Experience_lettre'] = experience_list
df['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df = df[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien',"Lieu de travail","Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

df


In [52]:
req = requests.get('https://www.novojob.com/cote-d-ivoire/societe-generale-sgci/offres-d-emplois/offre-d-emploi/ms_135277-charge-d-affaires-trade-finance-h-f' 
, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

In [63]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []
all_job_details = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# ...

for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
    bloc_bottom = offre.find_next('div', class_='bloc-bottom')
    intitules_list.append(offre.get_text().strip())
    entreprises_list.append(entreprise.get_text().strip())
    lien_list.append(category_links.index(category_link))

    # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
    pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
    pays = pays_info.find_parent().text.strip() if pays_info else None
    pays_list.append(pays)

    date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
    date = date_info.find_parent().text.strip() if date_info else None
    dates_list.append(date)

    # Ajout des colonnes pour le niveau du poste et l'expérience demandée
    niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
    niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

    # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
    match = re.match(r'(.+) \((.+)\)', niveau_text)

    if match:
        niveau_col, experience_col = match.groups()
    else:
        niveau_col, experience_col = None, None

    niveau_list.append(niveau_col)
    experience_list.append(experience_col)

    # Utilisation de Selenium pour cliquer sur l'élément
    offre_link = offre.find('a')
    if offre_link:
        offre_link = offre_link.get('href')
        driver.get(offre_link)
        time.sleep(2)  # Attendez que la page se charge après le clic

        # Utilisez BeautifulSoup pour extraire les détails de l'offre d'emploi à partir de la nouvelle page
        soup_details = BeautifulSoup(driver.page_source, 'html.parser')

        job_details = {}

        # Extracting job details
        details_section = soup_details.find('ul', class_='text-small')
        if details_section:
            for li in details_section.find_all('li', class_='row-fluid'):
                key = li.find('span', class_='span4').text.strip()
                value = li.find('span', class_='span8').text.strip()
                job_details[key] = value

        # Extracting the provided text
        description_section = soup_details.find('div', class_='spaced details-description')
        if description_section:
            provided_text = description_section.text.strip()
            job_details['Provided Text'] = provided_text

        # Add job details to the list
        all_job_details.append(job_details)

        # Go back to the previous page
        driver.back()
        time.sleep(2)  # Wait for the previous page to load

# ...


# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les offres d'emploi en DataFrame
df = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Intitule'] = intitules_list
df['Entreprise'] = entreprises_list
df['Pays'] = pays_list
df['Date'] = dates_list
df['Niveau'] = niveau_list
df['Experience_lettre'] = experience_list
df['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df = df[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

# Afficher le DataFrame
df


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien


In [ ]:
soup_details = BeautifulSoup(driver.page_source, 'html.parser')

In [64]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []
all_job_details = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

        # Utilisation de Selenium pour cliquer sur l'élément
        offre_link = offre.find('a')
        if offre_link:
            offre_link = offre_link.get('href')
            driver.get(offre_link)
            time.sleep(2)  # Attendez que la page se charge après le clic

            # Utilisez BeautifulSoup pour extraire les détails de l'offre d'emploi à partir de la nouvelle page
            soup_details = BeautifulSoup(driver.page_source, 'html.parser')

            job_details = {}

            # Extracting job details
            details_section = soup_details.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_details.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Add job details to the list
            all_job_details.append(job_details)

            # Go back to the previous page
            driver.back()
            time.sleep(2)  # Wait for the previous page to load

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les offres d'emploi en DataFrame
df = pd.DataFrame(all_job_details)

# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Intitule'] = intitules_list
df['Entreprise'] = entreprises_list
df['Pays'] = pays_list
df['Date'] = dates_list
df['Niveau'] = niveau_list
df['Experience_lettre'] = experience_list
df['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins


# Afficher le DataFrame
df


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Superviseur Marketing Opérationnel,KOMIAN AI,Côte d'ivoire,05 Janvier,Responsable d'équipe,3 à 5 ans,0
1,Stagiaire Professionnel Assistant (e) Marketing,Entreprise anonyme,"Abidjan, Côte d'ivoire",04 Janvier,Débutant / Junior,Moins d’un an,0
2,BTL Specialist,Exceliam,"Abidjan, Côte d'ivoire",28 Décembre 2023,Confirmé / Expérimenté,1 à 2 ans,0
3,Stagiaire Data Scientist,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",01 Décembre 2023,Jeune diplômé,Moins d’un an,0
4,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,0
5,Community Manger,KOMIAN AI,Côte d'ivoire,10 Décembre 2023,Confirmé / Expérimenté,3 à 5 ans,0
6,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0
7,Un(e) Commercial,SI’CLEAN PRESSING,Côte d'ivoire,09 Novembre 2023,Débutant / Junior,1 à 2 ans,0
8,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
9,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0


In [67]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Liste des liens pour chaque catégorie
category_links = [
    'https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q=marketing',
]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []
all_job_details = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()


# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('h2', class_='ellipsis row-fluid')
    entreprises = soup.find_all('h6', class_='ellipsis')
    niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    # Utilisation de Selenium pour cliquer sur l'élément
        offre_link = offre.find('a')
        if offre_link:
            offre_link = offre_link.get('href')
            driver.get(offre_link)
            time.sleep(2)  # Attendez que la page se charge après le clic

            # Utilisez BeautifulSoup pour extraire les détails de l'offre d'emploi à partir de la nouvelle page
            soup_details = BeautifulSoup(driver.page_source, 'html.parser')

            job_details = {}

            # Extracting job details
            details_section = soup_details.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_details.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Add job details to the list
            all_job_details.append(job_details)

            # Go back to the previous page
            driver.back()
            time.sleep(2)  # Wait for the previous page to load
# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les offres d'emploi en DataFrame
df = pd.DataFrame(all_job_details)  


IndentationError: unexpected indent (3364037873.py, line 41)